#Продвинутый Python, ДЗ-7

Правила игры:

В домашке 1 задача. Суммарно за дз можно получить 100 баллов, что равняется 10 баллам

Дедлайн - 7 дней после выдачи дз. Необходимо залить решеннный ноутбук в github и прислать ссылку в Anytask (без выполнения любого из пунктов работа проверяться не будет)

Оценка за ДЗ складывается следующим образом:

* наличие полного результата - 10 баллов

* наличие только 1 запроса - -4 балла

* отсутствие фильтрации - -2 балла

* запросы только с первой страницы - -2 балла

* не все поля вытащены - -2 балла

В качестве результата добавляйте в репозиторий полностью папку с пауком и всеми необходимыми файлами

**N.B.** Steam спокойно парсится без посторонних инструментов обхода блокировок

## Задание

Парсим [Steam](https://store.steampowered.com/) с помощью Scrapy!

Необходимо выбрать 3 запроса (например, "инди", "стратегии", "minecraft", на каждый запрос должно быть минимум 90 результатов), и вытащить по ним:

* Все игры на первых 2 страницах (найдите, каким образом отображаются страницы)

И для каждой игры вытащить:

* Название

* Категорию (весь путь, за исключением Все игры и самого названия игры)

* Число всех обзоров и общая оценка

* Дата выхода

* Разработчик

* Метки (тэги) игры

* Цена

* Доступные платформы

Сохраните игры в формате json, оставив только игры, выпущенные после 2000 года

Чтобы было проще, в репозитории с ДЗ выложены скрины, где можно посмотреть, где и какие поля находятся



In [ ]:
!pip install scrapy
import scrapy
from scrapy.crawler import CrawlerProcess
import json
import logging

In [2]:
PAGES_NUM = 2
# 19 = action
# 492 = indie
# 599 = simulators
STEAM_CATEGORIES = [492,19,599]

YEAR_NEWER_THAN = 2000

In [3]:
class JsonWriterPipeline(object):

    def open_spider(self, spider):
        self.file = open('result.json', 'w')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        if (int(item['published'].split()[-1]) >= YEAR_NEWER_THAN):
            line = json.dumps(dict(item)) + "\n"
            self.file.write(line)
        return item

class SteamGameItem(scrapy.Item):
    title = scrapy.Field()
    price = scrapy.Field()
    published = scrapy.Field()
    platforms = scrapy.Field()
    breadcrumb = scrapy.Field()
    developer = scrapy.Field()
    reviews = scrapy.Field()
    tags = scrapy.Field()

In [4]:
class SteamSpider(scrapy.Spider):
    name = "steam"
    start_urls = [
        'https://store.steampowered.com/search?tags=%d&page=%d' % (c,n) for c in STEAM_CATEGORIES for n in range(1,PAGES_NUM+1)
    ]
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1},
        'FEED_FORMAT':'json',
        'FEED_URI': 'quoteresult.json',
        'AUTOTHROTTLE_ENABLED': True,
        'AUTOTHROTTLE_TARGET_CONCURRENCY': 4.0,
        'HTTPCACHE_ENABLED': True
    }

    def parse(self, response):
        divs = response.css(".responsive_search_name_combined")
        for div in divs:
            item = SteamGameItem()
            link_to_game = div.xpath('../@href').get()
            item['title'] = div.css('span.title ::text').get()
            item['published'] = div.css('div.search_released ::text').get().strip()
            item['price'] = div.css('div.discount_final_price ::text').get()
            item['platforms'] = ['%s' % pl.join(pl.split(' ')[1:]) for pl in div.css('span.platform_img ::attr("class")').getall()]
            yield response.follow(url=link_to_game,callback=self.parse_game,meta = {'item':item})

    def parse_game(self, response):
        item = response.meta['item']
        item['breadcrumb'] = [_ for _ in [_.replace('>','').strip() for _ in response.css('div.breadcrumbs > div.blockbg ::text').getall()] if _ != ''][1:-1]
        item['developer'] = [_ for _ in [_.replace(',','').strip() for _ in response.css('div.summary#developers_list ::text').getall()] if _ != '']
        item['tags'] = [_ for _ in [_.replace('+','').strip() for _ in response.css('div.popular_tags ::text').getall()] if _ != '']
        item['reviews'] = response.css('meta[itemprop="reviewCount"]::attr(content)').get()
        yield item

In [ ]:
process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})
process.crawl(SteamSpider)
process.start()